NLP中最流行的模型

最核心的部分，多头注意力模块

将embedding分为三个数据，Q，K，V输入到多头注意力模块


一个多头注意力模块：
+ 经过多头注意力模块，得到一个输出，与没有经过的embedding相加，得到一个输出，并且做标准化
+ 然后经过一个全连接层，得到一个输出，与上面相同，相加并标准化

多个多头注意力模块组成encoder，输入是数据集中的输入数据

decoder部分像是一个新的没有全连接层的 masked多头注意力模块 加上一个完整的多头注意力模块
+ 没有全连接层的多头注意力模块为后面完整的注意力模块提供 Q 的表示
+ encoder为完整的注意力模块提供 K，V 的表示
+ mask 保证了当前位置的output只能看到在他位置之前的input

多个上述的复合多头注意力模块组成decoder，输入是数据集中的输出数据

encoder和decoder接收的输入有位置编码和embedding的concat

+ 相比起GRU，LSTM，transformer的左右操作都是并行完成的，

+ 这种并行也导致了需要用到mask，不然就只是学到了错位的映射关系，而不是根据前文预测

+ 多头注意力中的头指的是多个attention，就是将输入的向量分成多少份，并行计算？

+ Layernorm 和 Batchnorm： Layernorm是针对每个example自己做normalization，Batchnorm是每个Batch中的所有example一起做normalization，所以layernorm的计算量更大些

+ 编码器的任务是在每个位置学习到序列中各个标记之间的关系，最终为解码器提供一个完整的表示。解码器再根据这个表示和目标序列去预测下一个标记。

In [1]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()   # 这一行是为了调用父类的构造函数，不然会报错
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        
        assert (self.head_dim * heads == embed_size), "Embed size needs to be div by heads"   # 这一行是为了检查embed_size是否能被heads整除
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)   # 将多头的结果拼接起来，所以这一行的输入是heads * self.head_dim，输出是embed_size

    def forward(self, values, keys, query, mask):
        N = query.shape[0]   # 这一行是为了获取batch size, 输入数据的条数
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]   # 这一行是为了获取每个输入序列的长度

        # Split the embedding into self.heads pieces   # 这一行是为了将输入数据拆分成多个头，分别进行计算
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)   # 这一行是为了将value映射到head_dim维度的空间   # （N, value_len, heads, head_dim）
        keys = self.keys(keys)   # 这一行是为了将key映射到head_dim维度的空间   # （N, key_len, heads, head_dim）
        queries = self.queries(queries)   # 这一行是为了将query映射到head_dim维度的空间   # （N, query_len, heads, head_dim）
        
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])   # 这一行是为了计算query和key的点积，einsum函数是张量的乘法，nqhd表示N个序列，query的长度，head的数量，head的维度，nkhd表示N个序列，key的长度，head的数量，head的维度，nhqk表示head的数量，query的长度，key的长度，这里的q和k是query和key的缩写
        # queries shape: (N, query_len, heads, head_dim)
        # keys shape: (N, key_len, heads, head_dim)
        # energy shape: (N, heads, query_len, key_len)   # 得到的结果是query_len个单词和key_len个单词的点积，每个值代表单词间注意力的权重

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))   # 这一行是为了将mask中为0的位置的值替换成负无穷，这样在softmax之后就会变成0

        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3)   # 这一行是为了计算softmax，dim=3表示在key_len这个维度上进行softmax，即query_len个单词对key_len个单词的注意力权重，每个query单词对key_len个单词的注意力权重之和为1
        # 这里的self.embed_size ** (1/2)是为了防止点积过大，导致softmax之后的值过小(保证数值稳定性)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(N, query_len, self.heads * self.head_dim)   # reshape是将多头的结果又拼接起来
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, head_dim)
        # out shape: (N, query_len, heads, head_dim)
        # key_len和value_len是相等的，因为他们是相同的输入

        out = self.fc_out(out)
        return out
    

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),   # 这一行是为了将输入数据映射到更高维度，这样可以更好的学习特征
            nn.ReLU(),   # 这一行是为了引入非线性
            nn.Linear(forward_expansion * embed_size, embed_size)   # 这一行是为了将数据映射回原来的维度
        )   # 全连接层前后维度没有变化

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)   # 这一行是为了计算多头注意力，这里调用了SelfAttention类的forward函数，因为nn.Module类中已经定义了forward函数，所以当实例化一个nn.Module类的子类的时候，就会自动调用这个forward函数
        
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out
    
class Encoder(nn.Module):
    def __init__(self,
                 src_vocab_size,   # 输入数据的词汇表大小
                 embed_size,   # 词嵌入的维度
                 num_layers,   # 编码器的层数
                 heads,   # 多头注意力的头数
                 device,   # 训练设备
                 forward_expansion,   # 前向传播的扩展
                 dropout,   # dropout的概率
                 max_length,   # 输入数据的最大长度，句子的长度
                 ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)   # 将所有单词都映射到embed_size维度的空间
        self.position_embedding = nn.Embedding(max_length, embed_size)   # 将所有位置都映射到embed_size维度的空间
        
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout,
                    forward_expansion,
                )
                for _ in range(num_layers)   # 这一行是为了生成num_layers个TransformerBlock
            ]
        )
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        N, seq_length = x.shape   # 这一行是为了获取batch size和序列长度
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)   # 这一行是为了生成位置编码，这里的expand是为了将位置编码扩展到和输入数据一样的维度，to是为了将数据放到指定的设备上
        
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))   # 这一行是为了将输入数据和位置编码相加，这样就可以将位置信息和单词信息结合起来   # self.word_embedding不需要embed_size参数，因为在初始化的时候已经指定了
        
        for layer in self.layers:
            out = layer(out, out, out, mask)   # Q, K, V 都是out，因为这里是自注意力机制，即自己对自己进行注意力计算
            return out
        
class DecoderBlock(nn.Module):   # 一个decoder模块
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):   # src_mask是为了遮掩源数据（不计算为了填充位置对齐的位置），trg_mask是为了遮掩目标数据
        attention = self.attention(x, x, x, trg_mask)   # 这一行是为了计算目标数据的自注意力
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)   # 这里用src_mask是因为这里是对源数据的value,key和目标数据的query进行注意力计算
        return out
    
class Decoder(nn.Module):
    def __init__(self,
                 trg_vocab_size,   # 输出数据的词汇表大小
                 embed_size,   # 词嵌入的维度
                 num_layers,   # 解码器层数
                 heads,   # 多头注意力的头数
                 forward_expansion,   # 前向传播的扩展
                 dropout,   # dropout的概率
                 device,   # 训练设备
                 max_length,   # 输入数据的最大长度，句子的长度
                 ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        
        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device) for _ in range(num_layers)
            ]
        )

        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout(self.word_embedding(x) + self.position_embedding(positions))
        
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)
            
        out = self.fc_out(x)
        return out


class Transformer(nn.Module):
    def __init__(
            self,
            src_vocab_size,
            trg_vocab_size,
            src_pad_idx,   # 源数据的填充位置
            trg_pad_idx,   # 目标数据的填充位置
            embed_size=256,
            num_layers=6,
            forward_expansion=4,
            heads=8,
            dropout=0,
            device="cuda",
            max_length=100,
    ):
        super(Transformer, self).__init__()
        
        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)   # 这一行是为了生成源数据的mask，与src_pad_idx相等的部分为0，不等的部分为1，unsqueeze是为了增加维度，变成(N, 1, 1, src_len)，每一条数据的每个单词都有一个mask(一个0或1)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)
    
    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(N, 1, trg_len, trg_len)   # 这一行是为了生成目标数据的mask，tril是生成下三角矩阵，ones是生成全1矩阵，expand是为了将矩阵扩展到(N, 1, trg_len, trg_len)，这样每一条数据的每个单词都有一个mask
        # (N, 1, trg_len, trg_len)
        return trg_mask.to(self.device)
    
    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

In [2]:
if __name__ == "__main__":
    print(torch.cuda.is_available())
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10   # 单词词汇表的大小（本例子中是0-9）
    trg_vocab_size = 10
    
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(device)
    out = model(x, trg[:, :-1])   # trg[:, :-1]是为了去掉最后一个单词，因为最后一个单词是填充位置
    print(out.shape)   # torch.Size([2, 7, 10])，2是batch size，7是输出数据的长度，10是输出数据的词汇表大小

True
torch.Size([2, 7, 10])


In [4]:
out   # 输出的每个值都是预测的单词的概率，我们可以通过argmax函数找到概率最大的单词，即预测的单词
# 找到概率最大的单词
print(out.argmax(2))   # tensor([[1, 7, 4, 3, 5, 9, 2], [1, 5, 6, 2, 4, 7, 6]], device='cuda:0')，这里的结果是预测的单词
# trg是（2,8)的，去掉最后一个单词后是（2,7），所以预测的单词也是（2,7）的，所以预测对应的是输入的前7个单词，前七个单词组成七个句子

tensor([[0, 4, 0, 0, 8, 8, 9],
        [0, 8, 8, 8, 4, 4, 2]], device='cuda:0')
